# data processing
refactored to df

## imports

In [41]:
%reload_ext autoreload
%autoreload 2

In [42]:
import sys
import os
from pathlib import Path
import configparser
config = configparser.ConfigParser()
config.read_file(open('privateconfig'))
resdir = Path(config['Datafolder']['data'])
workdir = Path(config['Codefolder']['workspace'])
os.chdir(workdir)

In [43]:
# analysis
from scipy.io import loadmat
from sklearn.decomposition import FastICA
from sklearn.datasets import make_regression
from sklearn.model_selection import KFold
from sklearn.linear_model import LassoCV, Lasso
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr


In [44]:
# misc
import pickle
from collections import defaultdict

In [45]:
# task
from env_config import Config
from firefly_task import ffacc_real
# from monkey_functions import *
# from InverseFuncs import *
from stable_baselines3 import TD3
import torch


In [46]:
from neural_plot_ult import *
import time
tic=time.time()
import warnings
warnings.filterwarnings('ignore')

# Pre IRC

convert the mat data file (with neural data) into (states, actions, tasks) for IRC.


## prepare

In [47]:
# const
bin_size = 17  # how many bin of DT. about 0.1 s
# num_bins = 24  # how many bins to use. use 2.4 s and discard the long trials.
# monkey_height = 10
DT = 0.006  # DT for raw data
# reward_boundary = 65
areas = ['PPC', 'PFC', 'MST']
worldscale = 200


m = 'm51'
folder = 'm51_mat_ruiyi'
dens = [0.0001, 0.0005, 0.001,  0.005]

locals().update({m: {}})
figure_path = resdir/'figures'
# datapaths = [i for i in Pa1th(resdir/'mat_ruiyi').glob(f'{m}*.mat')]
datapaths = [i for i in Path(resdir/folder).glob(f'{m}*.mat')]
session=[int(a.stem[-2:]) for a in datapaths]


## from raw data file: task relavent variables and neural

In [48]:
# load raw data
df=pd.DataFrame() 
for idx, datapath in enumerate(datapaths):
    if datapath.stem[-1].isalpha():
        continue
    data = loadmat(datapath)
    eval(m)[datapath.stem] = data

    # df
    sessdf=pd.DataFrame() 
    sessdf['trial']=np.arange((len(data['trials_behv'][0])))
    sessdf['session']=int(datapath.stem[(datapath.stem).find('s')+1:])
    df=pd.concat([df, sessdf])

In [49]:
trial_behv['logical']['firefly_fullON'][0][0][0][0]

0

In [50]:
sessdata = defaultdict(list)

for key, data in eval(m).items():
    sess = int(key[-2:])
    if key[-1].isalpha():
        continue

    trials_behv = data['trials_behv'][0]
    trials_units = data['units'][0]
    units_area = np.array([v[0] for v in trials_units['brain_area']])
    trials_error = []
    trials_error_sign = []
    trials_target_angle = []
    trials_target_distance = []

    for trial_idx, trial_behv in enumerate(trials_behv):
        trial_ts = trial_behv['continuous']['ts'][0][0].reshape(-1)
        t_mask = (trial_ts > 0) & (
            ~np.isnan(trial_behv['continuous']['ymp'][0][0].reshape(-1)))
        t_mask &= trial_ts < trial_behv['events']['t_stop'][0][0].reshape(-1)
        if t_mask.sum() > 0:
            # remove the first data point to avoid downsample error
            t_mask[np.where(t_mask == True)[0][0]] = False

        # task varaibles from data
        mx = trial_behv['continuous']['xmp'][0][0][t_mask]
        my = trial_behv['continuous']['ymp'][0][0][t_mask]
        fx = trial_behv['continuous']['xfp'][0][0][t_mask]
        fy = trial_behv['continuous']['yfp'][0][0][t_mask]
        eye_hor_theta = trial_behv['continuous']['yre'][0][0][t_mask] # use yle for left eye.
        eye_ver_theta = trial_behv['continuous']['zre'][0][0][t_mask]
        # print(len(eye_hor_theta), len(eye_ver_theta), len(mx))
        mv = trial_behv['continuous']['v'][0][0][t_mask].reshape(-1, 1)
        mw = trial_behv['continuous']['w'][0][0][t_mask].reshape(-1, 1)

        # some adjustment for screen distance
        sx = np.ones_like(fx)
        sy = np.ones_like(fy)
        if my.size > 0:
            fx = np.ones_like(fx) * fx[0]
            fy = np.ones_like(fy) * fy[0]
            sx *= mx[-1]
            sy *= my[-1]
            my = my + 30
            fy = fy + 30
            sy = sy + 30

        # some thing coudl be removed from there.
        dx = fx - mx; dy = fy - my
        rel_dist = np.sqrt(dx**2 + dy**2); rel_ang = np.rad2deg(np.arctan2(dy, dx))
        rel_dist_stop = np.sqrt((sx - mx)**2 + (sy - my)**2)
        abs_dist = np.sqrt(mx**2 + my**2); abs_ang = np.rad2deg(np.arctan2(my, mx))
        body_theta = np.deg2rad(np.cumsum(mw*-1) * DT + 90)
        body_x, body_y = mx.reshape(-1), my.reshape(-1)


        # skip bad trial
        if t_mask.sum() * DT > 3.5 or t_mask.sum() * DT < 0.6 or mv.max() < 50 or \
                abs_dist[-1] < np.sqrt(fx**2 + fy**2)[-1] * 0.3:
            continue

        # errors
        if my.size > 0:
            trials_error.append(rel_dist[-1][0])
            trials_error_sign.append(rel_dist[-1][0])
            trials_target_angle.append(
                np.rad2deg(np.arctan2(fy, fx))[-1][0] - 90)
            trials_target_distance.append(np.sqrt(fx**2 + fy**2)[-1][0])
            d1 = np.sqrt(fx**2 + fy**2)
            r1 = (fx**2 + fy**2) / (2*fx)
            radian1 = 2 * r1 * np.arcsin(d1 / (2 * r1))
            d2 = np.sqrt(mx**2 + my**2)
            r2 = (mx**2 + my**2) / (2*mx + 1e-8)
            radian2 = 2 * r2 * np.arcsin(d2 / (2 * r2 + 1e-8))
            sign = np.ones_like(rel_dist)
            sign[radian2 < radian1] = -1
            rel_dist = sign * rel_dist
            trials_error_sign[-1] = rel_dist[-1][0]
        else:
            trials_error.append(np.nan)
            trials_error_sign.append(np.nan)
            trials_target_angle.append(np.nan)
            trials_target_distance.append(np.nan)

        target_variable = np.hstack([rel_dist, rel_ang, abs_dist, abs_ang,
                                     eye_hor_theta, eye_ver_theta, 
                                     fx, fy, mx, my, mv, mw])

        target_variable_ds = downsample(target_variable, bin_size=bin_size)

        (rel_dist, rel_ang, abs_dist, abs_ang,
         eye_hor_theta, eye_ver_theta,
         fx, fy, mx, my, mv, mw) = zip(*target_variable_ds)
        body_theta = -np.deg2rad(np.cumsum(mw) * 0.1 - 90)
        body_x, body_y = np.array(mx).reshape(-1), np.array(my).reshape(-1)
        latent_ff_hori, latent_ff_vert = convert_location_to_angle(abs(np.array(rel_dist)).reshape(-1), np.array(fx).reshape(-1), np.array(fy).reshape(-1),
                                                                   np.array(body_theta), np.array(
                                                                       body_x), np.array(body_y),
                                                                   np.array(eye_hor_theta).reshape(-1), np.array(eye_ver_theta).reshape(-1), remove_pre=False)

        # df
        sessdata['session'].append(int(sess))
        sessdata['trial'].append(trial_idx)
        sessdata['fullon'].append(trial_behv['logical']['firefly_fullON'][0][0][0][0])
        sessdata['density'].append(
            (trial_behv['prs'][0][0]['floordensity'].item()))
        # sessdata['rel_dist'].append(rel_dist)
        # sessdata['rel_ang'].append(rel_ang)
        # sessdata['abs_dist'].append(abs_dist)
        # sessdata['abs_ang'].append(abs_ang)
        sessdata['eye_hori'].append(eye_hor_theta)
        sessdata['eye_vert'].append(eye_ver_theta)
        sessdata['ff_hori'].append(latent_ff_hori.reshape(-1))
        sessdata['ff_vert'].append(latent_ff_vert.reshape(-1))
        sessdata['fx'].append(fx)
        sessdata['fy'].append(fy)
        sessdata['mx'].append(mx)
        sessdata['my'].append(my)
        sessdata['mv'].append(mv)
        sessdata['mw'].append(mw)

        # neural
        activities = []  # activities for all neurons for 1 trial. shape: ts, neurons
        for trials_unit in trials_units:
            fire_ts = trials_unit['trials'][0][trial_idx][0].reshape(-1)
            if fire_ts.size > 0 and fire_ts[-1] >= trial_ts[-1]:
                fire_ts = fire_ts[:-1]
            activity = np.zeros_like(trial_ts)
            bin_indices = np.digitize(fire_ts, trial_ts)
            unique_bins, bin_counts = np.unique(
                bin_indices, return_counts=True)
            activity[unique_bins] = bin_counts
            activities.append(activity)

        activities = np.vstack(activities).T   # time * unit
        activities = activities[t_mask]
        activities = gaussian_filter1d(
            activities, sigma=4, axis=0)  # neural for each trial
        activityds = downsample(activities, bin_size=bin_size)

        for area in areas:
            area_mask = [v in area for v in units_area]
            if sum(area_mask) == 0:
                activity_ = np.nan
            else:
                activity_ = activityds[:, area_mask]  # area activity
            sessdata[area].append(activity_)

    sessdata['error'] += (trials_error)
    sessdata['error_sign'] += (trials_error_sign)
    sessdata['target_angle'] += (trials_target_angle)
    sessdata['target_distance'] += (trials_target_distance)

tmp = pd.DataFrame(sessdata)
df = pd.merge(df, tmp, on=['trial', 'session'], how='inner')

In [51]:
sessdf=df[df.session==38]
trialidx=np.random.choice(sessdf.trial.unique())
[len(a) for a in [sessdf.iloc[trialidx].eye_hori, sessdf.iloc[trialidx].eye_vert]]

[16, 16]

## IRC input data (state, action, task)

In [52]:
sessdata=defaultdict(list)

for sess in session:
    states, actions, tasks=[],[],[]
    sessdf=df[df.session==sess]
    trial_idces=sessdf.trial

    for trial_idx in trial_idces:
        trialdf=sessdf[sessdf.trial==trial_idx]
        trialdata=trialdf.iloc[0]
        # task
        taskx = (trialdata.fx[0] - trialdata.mx[0]).astype('float32'); tasky = (trialdata.fy[0] - trialdata.my[0]).astype('float32')
        tasks.append([tasky/worldscale,taskx/worldscale])
        # actions
        trialaction=np.stack([trialdata.mv,trialdata.mw]).T
        trialaction[:,0]=trialaction[:,0]/worldscale # v need reduce scale
        trialaction[:,1]=trialaction[:,1]/180*pi
        actions.append(trialaction.astype('float32'))

        # states from run the actions
        px, py, heading, v, w = 0,0,0,0,0
        log=[]
        for a in trialaction:
            px, py, heading, v, w=state_step2(px, py, heading, v, w, a, dt=0.1,userad=True)
            log.append([px, py, heading, v, w])
        px, py, heading, v, w=state_step2(px, py, heading, v, w, a, dt=0.1,userad=True)
        log.append([px, py, heading, v, w])
        trialstates=np.array(log)[1:]
        
        states.append(trialstates.astype('float32'))

        sessdata['session'].append(sess)
        sessdata['trial'].append(trial_idx)
        sessdata['state'].append(trialstates.astype('float32'))  
        sessdata['action'].append(trialaction.astype('float32'))  
        sessdata['task'].append([tasky/worldscale,taskx/worldscale])  

tmp=pd.DataFrame(sessdata)
df = pd.merge(df, tmp, on=['trial','session'], how='inner')

## Compute belief 

In [53]:
from InverseFuncs import monkeyloss_
from neural_plot_ult import process_inv

In [54]:
# model estimated likelihood (negative log likelihood)
torch.manual_seed(42)
arg = Config()

env = ffacc_real.FireFlyPaper(arg)
env.debug=True
phi = torch.tensor([[0.5],
                    [pi/2],
                    [0.001],
                    [0.001],
                    [0.001],
                    [0.001],
                    [0.13],
                    [0.001],
                    [0.001],
                    [0.001],
                    [0.001],
                    ])

agent_ = TD3.load(workdir/'trained_agent/paper')
agent = agent_.actor.mu.cpu()

def lltrial(state, action, task,samples=5):
    with torch.no_grad():
        return monkeyloss_(agent, action, task, phi, finaltheta, env, action_var=0.01, num_iteration=1, states=state, samples=samples, gpu=False).item()
    

# irc_ll={}
# for key, data in eval(m).items():
#     if not key.endswith('irc'):
#         continue
#     states, actions, tasks=eval(m)[key]

#     sessionll=[]
#     for i in range(len(states)):
#         state, action, task=[states[i]], [actions[i]], [tasks[i]]
#         ll=lltrial(state, action, task)
#         sessionll.append(ll)
#         if i%100==0: print(i)
#     irc_ll[key+'_ll']=sessionll

# notify('all done')

In [55]:
thetas={}
for idensity in range(4):
    datapath = Path(resdir/f'{m}_mat_ruiyi/preirc_den_{idensity}')
    savename = datapath.parent/(f'{m}_{idensity}'+datapath.name)
    invfile=savename
    print(datapath.name)
    finaltheta, finalcov, err = process_inv(
        invfile, removegr=False, usingbest=False)
    print(finaltheta[:4])
    # finaltheta[0]=1
    # finaltheta[1]=1.3
    # finaltheta[1]=0.5
    # finaltheta[1]=0.2
    thetas[idensity]=finaltheta


preirc_den_0
/Users/yc/Documents/lab_data/m51_mat_ruiyi/m51_0preirc_den_0
using ind:  -1 final logll :  16.993248803274973
tensor([[0.9924],
        [0.7905],
        [0.7104],
        [0.1499]])
preirc_den_1
/Users/yc/Documents/lab_data/m51_mat_ruiyi/m51_1preirc_den_1
using ind:  -1 final logll :  15.665690626416888
tensor([[1.1921],
        [0.8381],
        [0.6041],
        [0.1469]])
preirc_den_2
/Users/yc/Documents/lab_data/m51_mat_ruiyi/m51_2preirc_den_2
using ind:  -1 final logll :  16.006959642682755
tensor([[1.1985],
        [0.9138],
        [0.5796],
        [0.1894]])
preirc_den_3
/Users/yc/Documents/lab_data/m51_mat_ruiyi/m51_3preirc_den_3
using ind:  -1 final logll :  15.557111876351494
tensor([[1.1422],
        [0.8885],
        [0.5519],
        [0.1930]])


In [56]:
# compute belief
today='0110'
skipblief=not(date.today().strftime("%m%d") == today)

if not skipblief: # compute new belief and save with today date
    all_beliefs, all_covs={},{}
    denslookup={0.0001:0, 0.0005:1, 0.001:2,  0.005:3}
    for key, data in eval(m).items():
        if not key.endswith('irc'):
            continue
        datakey=key[:-16] # something like m51s11
        dens=np.concatenate(eval(m)[datakey+'_dens']).reshape(-1)
        states, actions, tasks=eval(m)[key]
        beliefs, covs = [], []
        ntrial = 1
        # theta = finaltheta

        for ind in range(len(tasks)):
            if ind%100==0:
                print(ind)
            if len(actions[ind]) < 5:
                beliefs.append(np.nan)
                covs.append(np.nan)
            else:
                den=dens[ind]
                theta=thetas[denslookup[den]]
                _, _, ep_beliefs, ep_covs = run_trials(agent=agent, env=env, phi=phi, theta=theta, task=tasks[ind], ntrials=ntrial,
                                                    pert=None, given_obs=None, return_belief=True, given_action=actions[ind], given_state=states[ind])
                # beliefs.append(ep_beliefs[0]-ep_beliefs[0][0])
                init=torch.tensor(states[ind][0]).reshape(-1,1)
                beliefs.append(ep_beliefs[0]-ep_beliefs[0][0]+init) # make belief and state start at the same place
                covs.append(ep_covs[0])
                assert len(ep_beliefs[0]) == len(actions[ind])
        all_beliefs[f'{key[:-4]}_irc_belief']=beliefs
        all_covs[f'{key[:-4]}_irc_cov']=covs
    notify('done computing belief')
    today = date.today().strftime("%m%d") # mm/dd
    with open(resdir/f'{folder}/irc_belief_{today}','wb+') as f:
        pickle.dump((all_beliefs, all_covs),f)

else: # use the given day
    print(f'use computed belief from day {today}')
    with open(resdir/f'{folder}/irc_belief_{today}','rb') as f:
        all_beliefs, all_covs=pickle.load(f)

use computed belief from day 0110


## Compute likelihood
todo, need to do this trial by trial to take account of density.

In [57]:
today=1117
skipll=not(date.today().strftime("%m%d") == today)

def lltrial(state, action, task, finaltheta, samples=5):
    with torch.no_grad():
        return monkeyloss_(agent, action, task, phi, finaltheta, env, action_var=0.01, num_iteration=1, states=state, samples=samples, gpu=False).item()
    
if not skipll: # compute new ll and save with today date
    irc_ll={}
    for key, data in eval(m).items():
        if not key.endswith('irc'):
            continue
        states, actions, tasks=eval(m)[key]
        datakey=key[:-16] # something like m51s11
        dens=np.concatenate(eval(m)[datakey+'_dens']).reshape(-1)

        sessionll=[]
        for i in range(len(states)):
            state, action, task=[states[i]], [actions[i]], [tasks[i]]
            den=dens[i]
            ll=lltrial(state, action, task, thetas[denslookup[den]])
            sessionll.append(ll)
            if i%100==0: print(i)
        irc_ll[key+'_ll']=sessionll
    notify('all done')
    today = date.today().strftime("%m%d") # mm/dd
    with open(resdir/f'{folder}/irc_ll_{today}','wb+') as f:
        pickle.dump(irc_ll,f)

else: # use the given day
    print(f'use computed likelihood from day {today}')
    with open(resdir/f'{folder}/irc_ll_{today}','rb') as f:
        irc_ll=pickle.load(f)


use computed likelihood from day 1117


In [58]:
# temp, just load the belief and likelihood. TODO, compute for df version
sessdata=defaultdict(list)

for sess in session:
    belief_key=f'm51s{sess}Ydownsampled_irc_belief'
    beliefcov_key=f'm51s{sess}Ydownsampled_irc_cov'
    likelihood_key=f'm51s{sess}Ydownsampled_irc_ll'
    sessdata['belief']+=[a.numpy()[:,:,0] for a in all_beliefs[belief_key]]
    sessdata['uncertainty']+=[a.numpy() for a in all_covs[beliefcov_key]]
    sessdata['likelihood']+=irc_ll[likelihood_key]


# TODO, merge instead of just asssign to be robust
# tmp=pd.DataFrame(sessdata)
# df = pd.merge(df, tmp, on=['trial','session'], how='inner')

# tmp
df['belief']=sessdata['belief']
df['uncertainty']=sessdata['uncertainty']
df['likelihood']=sessdata['likelihood']
df

,trial,session,fullon,density,eye_hori,eye_vert,ff_hori,ff_vert,fx,fy,...,error,error_sign,target_angle,target_distance,state,action,task,belief,uncertainty,likelihood
0,5,38,0,0.0010,"(8.602277, 6.598316, -2.5812783, -3.7105117, -...","(-4.4160075, -4.2855043, -3.4631371, -3.708253...","[-3.9195764, -5.21961, -6.6186395, -7.7913127,...","[-2.07629366923854, -2.2353554820547705, -2.42...","(-14.091859, -14.091859, -14.091859, -14.09185...","(284.28226, 284.28226, 284.28226, 284.28226, 2...",...,74.410896,-74.410896,2.837814,284.631348,"[[0.09495927, 0.0008101619, 0.017062943, 0.949...","[[0.9496387, 0.17062943], [0.94951963, 0.17074...","[1.3773353576660157, -0.0708301830291748]","[[0.09495927, 0.00081016193, 0.017062943, 0.94...","[[[9.0249996e-05, 0.0, 0.0, 0.0, 0.0], [0.0, 0...",16.596571
1,6,38,1,0.0050,"(-0.3632372, -0.2045524, 6.1901956, 12.163926,...","(-0.35722262, -0.45224428, -2.9772847, -5.1802...","[11.616706, 13.094926, 14.745572, 14.814495, 1...","[-2.5141420128085863, -2.7418255235227575, -3....","(45.447224, 45.447224, 45.447224, 45.447224, 4...","(231.10182, 231.10182, 231.10182, 231.10182, 2...",...,30.782871,-30.782871,-11.125511,235.528152,"[[0.08923148, -0.000103924016, -0.0023293118, ...","[[0.8923156, -0.023293117], [0.91945374, -0.07...","[1.115883331298828, 0.22691679000854492]","[[0.08923147, -0.000103923994, -0.0023293116, ...","[[[9.0249996e-05, 0.0, 0.0, 0.0, 0.0], [0.0, 0...",8.988084
2,7,38,0,0.0050,"(-2.1006086, -1.9773198, 7.058173, 10.111072, ...","(-2.515528, -2.6270876, -4.024856, -4.701935, ...","[11.439537, 12.727621, 14.1388855, 14.195187, ...","[-2.4657422557944657, -2.6161027569630813, -2....","(45.69279, 45.69279, 45.69279, 45.69279, 45.69...","(231.17816, 231.17816, 231.17816, 231.17816, 2...",...,7.206552,7.206552,-11.180519,235.650497,"[[0.04765145, -4.674053e-05, -0.0019617667, 0....","[[0.47651482, -0.019617667], [0.79956645, -0.1...","[1.1384558868408203, 0.22826662063598632]","[[0.04765145, -4.6740526e-05, -0.0019617665, 0...","[[[9.0249996e-05, 0.0, 0.0, 0.0, 0.0], [0.0, 0...",12.749125
3,8,38,0,0.0010,"(1.3335737, 1.3782939, 5.8996487, 17.797056, 1...","(-2.0928805, -2.0094526, -2.5867834, -4.203169...","[22.93508, 22.980186, 23.035349, 22.372557, 19...","[-1.9797700549715942, -1.9828353155082388, -1....","(112.8446, 112.8446, 112.8446, 112.8446, 112.8...","(265.4412, 265.4412, 265.4412, 265.4412, 265.4...",...,52.990173,-52.990173,-23.031357,288.431854,"[[1.1285858e-06, -3.931383e-12, -6.966977e-06,...","[[1.1285858e-05, -6.966977e-05], [-4.2427568e-...","[1.3319671630859375, 0.5639923477172851]","[[1.1285857e-06, -3.931383e-12, -6.966977e-06,...","[[[9.0249996e-05, 0.0, 0.0, 0.0, 0.0], [0.0, 0...",9.130776
4,9,38,0,0.0010,"(0.922414, 0.73104167, -6.88214, -11.191133, -...","(-1.342575, -1.4165788, -4.4917336, -6.288393,...","[-15.846026, -17.878063, -20.797728, -23.19515...","[-4.332660743157386, -4.7556852993166014, -5.4...","(-35.547142, -35.547142, -35.547142, -35.54714...","(129.60463, 129.60463, 129.60463, 129.60463, 1...",...,31.439251,-31.439251,15.337547,134.391083,"[[0.035438497, 7.758149e-05, 0.0043783667, 0.7...","[[0.3543861, 0.04378367], [0.71644366, 0.08136...","[0.6355034255981445, -0.17792604446411134]","[[0.035438493, 7.7581484e-05, 0.0043783667, 0....","[[[9.0249996e-05, 0.0, 0.0, 0.0, 0.0], [0.0, 0...",14.214041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,1497,43,0,0.0005,"(1.0314617, 1.9819208, 2.8835914, 3.38268, 3.6...","(-1.1344712, -1.3599807, -1.595985, -1.7848669...","[36.71874, 40.11445, 42.94797, 42.90087, 40.01...","[-1.7195042621778398, -1.797094465172341, -1.8...","(191.03563, 191.03563, 191.03563, 191.03563, 1...","(280.92453, 280.92453, 280.92453, 280.92453, 2...",...,35.039143,-35.039143,-34.216671,339.725189,"[[0.08575181, -0.0013048627, -0.030431125, 0.8...","[[0.85765046, -0.30431125], [0.88866675, -0.24...","[1.364395751953125, 0.9551200103759766]","[[0.08575181, -0.0013048628, -0.03043112

In [59]:
# unpack the belief state
df['bmx']=df.apply(lambda x:x.belief[:,1]*worldscale, axis=1)
df['bmy']=df.apply(lambda x:x.belief[:,0]*worldscale, axis=1)
df['heading']=df.apply(lambda x: x.state[:,2]*180/pi, axis=1)
df['belief_heading']=df.apply(lambda x: x.belief[:,2]*180/pi, axis=1)
df['timer']=df.apply(lambda x: np.arange(len(x.mx)), axis=1)
df['countdown']=df.apply(lambda x: np.flip(-np.arange(len(x.mx)), axis=0), axis=1)

In [60]:
# temp, just load the belief and likelihood. TODO, compute for df version
sessdata=defaultdict(list)

for sess in session:
    sessdf=df[df.session==sess]
    states, actions, tasks = sessdf.state.to_list(), sessdf.action.to_list(),sessdf.task.to_list()
    beliefs,covs=sessdf.belief.to_list(),sessdf.uncertainty.to_list()

    sess_latentff_hori, sess_latentff_vert = [], []
    for ep_beliefs, ep_covs, task in zip(beliefs, covs, tasks): # process for each trial
        mx, my, body_theta,  mv, mw = zip(*ep_beliefs)
        body_theta = -(np.cumsum(mw) * 0.1-pi/2)
        body_x, body_y = np.asarray(my).reshape(-1).astype('float') * \
            worldscale, np.asarray(mx).reshape(-1).astype('float')*worldscale

        fx, fy = task[1]*worldscale, task[0]*worldscale
        rel_dist = ((fx-body_x)**2+(fy-body_y)**2)**0.5
        hor_theta_, ver_theta_ = convert_location_to_angle(abs(np.array(rel_dist)).reshape(-1).astype('float'), np.array(fx).reshape(-1).astype('float'), np.array(fy).reshape(-1).astype('float'),
                                                           body_theta.astype('float'), body_x.astype(
                                                               'float'), body_y.astype('float'),
                                                           np.array(rel_dist).reshape(-1).astype('float'), # use the true eye positions to remove pre saccade movement and after overshooting eye movement
                                                           np.array(rel_dist).reshape(-1).astype('float'), DT=0.1, remove_pre=False, remove_post=False)
        # plt.plot(hor_theta_, ver_theta_, 'g')
        sess_latentff_hori.append(hor_theta_)
        sess_latentff_vert.append(ver_theta_)
    sessdata['belief_ff_hori']+=[a.reshape(-1) for a in sess_latentff_hori]
    sessdata['belief_ff_vert']+=[a.reshape(-1) for a in sess_latentff_vert]
    

# TODO temp need change to merge
df['belief_ff_hori']=sessdata['belief_ff_hori']
df['belief_ff_vert']=sessdata['belief_ff_vert']

In [61]:
# angle from start
def get_angle_from_start(row):
    return np.arctan2((np.array(row.my)), (np.array(row.mx)))

df['angle_from_start']=df.apply(get_angle_from_start, axis=1)

def get_belief_angle_from_start(row):
    return np.arctan2((np.array(row.bmy)), (np.array(row.bmx)))

df['belief_angle_from_start']=df.apply(get_angle_from_start, axis=1)


In [62]:
# # some thing can be moved out
# dx = fx - mx
# dy = fy - my
# rel_dist = np.sqrt(dx**2 + dy**2)
# rel_ang = np.rad2deg(np.arctan2(dy, dx))
# rel_dist_stop = np.sqrt((sx - mx)**2 + (sy - my)**2)
# abs_dist = np.sqrt(mx**2 + my**2)
# abs_ang = np.rad2deg(np.arctan2(my, mx))
# mw = -trial_behv['continuous']['w'][0][0][t_mask].reshape(-1)
# body_theta = np.deg2rad(np.cumsum(mw) * DT + 90)
# body_x, body_y = mx.reshape(-1), my.reshape(-1)
# latent_ff_hori, latent_ff_vert = convert_location_to_angle(abs(rel_dist).reshape(-1), fx.reshape(-1), fy.reshape(-1),
#                                                            body_theta, body_x, body_y,
#                                                            eye_hor_theta.reshape(-1), eye_ver_theta.reshape(-1), remove_pre=False, remove_post=False)


In [63]:
# check len
random_rows = df.sample(n=3)

# Iterate over the columns
for col in random_rows.columns:
    try:
        # Get the column values for the selected rows
        col_values = random_rows[col]
        # Calculate the length of each column value
        lengths = col_values.apply(lambda x: len(x))
        # Print column name and the length of each column value
        print(f"Column: {col}, Lengths: {lengths.tolist()}")
    except: continue


Column: eye_hori, Lengths: [25, 22, 15]
Column: eye_vert, Lengths: [25, 22, 15]
Column: ff_hori, Lengths: [25, 22, 15]
Column: ff_vert, Lengths: [25, 22, 15]
Column: fx, Lengths: [25, 22, 15]
Column: fy, Lengths: [25, 22, 15]
Column: mx, Lengths: [25, 22, 15]
Column: my, Lengths: [25, 22, 15]
Column: mv, Lengths: [25, 22, 15]
Column: mw, Lengths: [25, 22, 15]
Column: PPC, Lengths: [25, 22, 15]
Column: state, Lengths: [25, 22, 15]
Column: action, Lengths: [25, 22, 15]
Column: task, Lengths: [2, 2, 2]
Column: belief, Lengths: [25, 22, 15]
Column: uncertainty, Lengths: [25, 22, 15]
Column: bmx, Lengths: [25, 22, 15]
Column: bmy, Lengths: [25, 22, 15]
Column: heading, Lengths: [25, 22, 15]
Column: belief_heading, Lengths: [25, 22, 15]
Column: timer, Lengths: [25, 22, 15]
Column: countdown, Lengths: [25, 22, 15]
Column: belief_ff_hori, Lengths: [25, 22, 15]
Column: belief_ff_vert, Lengths: [25, 22, 15]
Column: angle_from_start, Lengths: [25, 22, 15]
Column: belief_angle_from_start, Lengths:

In [64]:
# check size
random_rows = df.sample(n=3)

# Iterate over the columns
for col in random_rows.columns:
    try:
        # Get the column values for the selected rows
        col_values = random_rows[col]
        # Calculate the length of each column value
        lengths = col_values.apply(lambda x: (x.shape))
        # Print column name and the length of each column value
        print(f"Column: {col}, Lengths: {lengths.tolist()}")
    except: continue

Column: ff_hori, Lengths: [(16,), (22,), (10,)]
Column: ff_vert, Lengths: [(16,), (22,), (10,)]
Column: PPC, Lengths: [(16, 94), (22, 112), (10, 94)]
Column: state, Lengths: [(16, 5), (22, 5), (10, 5)]
Column: action, Lengths: [(16, 2), (22, 2), (10, 2)]
Column: belief, Lengths: [(16, 5), (22, 5), (10, 5)]
Column: uncertainty, Lengths: [(16, 5, 5), (22, 5, 5), (10, 5, 5)]
Column: bmx, Lengths: [(16,), (22,), (10,)]
Column: bmy, Lengths: [(16,), (22,), (10,)]
Column: heading, Lengths: [(16,), (22,), (10,)]
Column: belief_heading, Lengths: [(16,), (22,), (10,)]
Column: timer, Lengths: [(16,), (22,), (10,)]
Column: countdown, Lengths: [(16,), (22,), (10,)]
Column: belief_ff_hori, Lengths: [(16,), (22,), (10,)]
Column: belief_ff_vert, Lengths: [(16,), (22,), (10,)]
Column: angle_from_start, Lengths: [(16,), (22,), (10,)]
Column: belief_angle_from_start, Lengths: [(16,), (22,), (10,)]


In [65]:
# test how to use df -----
sessdf=df[df.session==sess]
# a=np.concatenate(sessdf.belief_ff_hori.to_numpy())
# plt.plot(a[:333])
# a=np.concatenate(sessdf.ff_hori.to_numpy())
# plt.plot(a[:333])

## the varialbes we have:

In [66]:
# TODO, 
'''
there are few more var we want:
    error for belief
    state heading
    belief heading
    
'''
df.columns

Index(['trial', 'session', 'fullon', 'density', 'eye_hori', 'eye_vert',
       'ff_hori', 'ff_vert', 'fx', 'fy', 'mx', 'my', 'mv', 'mw', 'PPC', 'PFC',
       'MST', 'error', 'error_sign', 'target_angle', 'target_distance',
       'state', 'action', 'task', 'belief', 'uncertainty', 'likelihood', 'bmx',
       'bmy', 'heading', 'belief_heading', 'timer', 'countdown',
       'belief_ff_hori', 'belief_ff_vert', 'angle_from_start',
       'belief_angle_from_start'],
      dtype='object')

In [67]:
df.to_pickle(resdir/'m51df.pkl')
notify('saved')